# Conexión

In [30]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import pandas as pd
import numpy as np
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display
from sqlalchemy.engine import URL
import ast

In [31]:
USER = "slqleo"
PWD  = "G?,6(4(q$GMw"          # sin escaparla, URL.create se encarga
HOST = "45.169.253.115"       # o el host que realmente tenga la DB
PORT = 3306

srv_url = URL.create(
    drivername="mysql+pymysql",
    username=USER,
    password=PWD,
    host=HOST,
    port=PORT,
    query={"charset": "utf8mb4"},
)
engine_srv = create_engine(srv_url, pool_pre_ping=True)

with engine_srv.connect() as conn:
    server = conn.execute(text("SELECT @@hostname, @@port, @@version")).one()
    print("Conectado a:", server)

Conectado a: ('serverbd', 3306, '8.0.44-0ubuntu0.22.04.1')


# Usabilidad

In [32]:
with engine_srv.connect() as conn:
    usability_events = pd.read_sql(
        text("SELECT * FROM slq_main.usability_events"),
        conn
    )

usability_events.head(2)

,id,function_used,slq_company_id,sys_user_id,acl_role_id,created_at,updated_at
0,1,Visualización del detalle de un producto,168,2146.0,1.0,2025-03-26 21:10:48,2025-03-26 21:10:48
1,2,Visualización del detalle de un producto,168,2146.0,1.0,2025-03-26 21:11:08,2025-03-26 21:11:08


In [33]:
query = text("""
SELECT 
    u.function_used,
    u.created_at,
    u.sys_user_id AS usuario,
    e.nombre AS empresa
FROM slq_main.usability_events u
INNER JOIN slq_main.slq_empresas e 
    ON u.slq_company_id = e.id
LEFT JOIN slq_main.sys_users ur 
    ON u.sys_user_id = ur.id
WHERE 
    e.activo = 1
    AND ur.activo = 1
    AND ur.interno = 0
    AND e.nombre NOT REGEXP 'Mejora|academy|prueba|Demostración'
""")

with engine_srv.connect() as conn:
    df_usuarios = pd.read_sql(query, conn)

df_usuarios

,function_used,created_at,usuario,empresa
0,Acceso al módulo de productos,2025-03-27 08:01:32,26,CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR
1,Visualización del detalle de un producto,2025-03-27 08:02:05,26,CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR
2,Descarga de etiquetas,2025-03-27 08:02:32,26,CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR
3,Acceso al módulo de productos,2025-03-27 08:16:25,26,CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR
4,Visualización del detalle de un producto,2025-03-27 08:17:02,26,CAJA DE COMPENSACIÓN FAMILIAR COMPENSAR
...,...,...,...,...
151761,Ingreso al módulo de documentos,2025-12-04 17:41:54,3177,INSTITUTO COLOMBIANO DE NORMAS TÉCNICAS Y CERT...
151762,Ingreso a login,2025-12-05 07:28:53,3177,INSTITUTO COLOMBIANO DE NORMAS TÉCNICAS Y CERT...
151763,Ingreso al módulo de documentos,2025-12-05 07:28:58,3177,INSTITUTO COLOMBIANO DE NORMAS TÉCNICAS Y CERT...
151764,Descarga de documentos,2025-12-05 07:29:06,3177,INSTITUTO COLOMBIANO DE NORMAS TÉCNICAS Y CERT...


In [34]:
df_usuarios.to_csv("empresas.csv")

# Productos

In [35]:
query = text("""
WITH audit_crea AS (
    SELECT 
        a.auditable_id,
        COALESCE(a.usu_id, a.user_id) AS user_id,
        ROW_NUMBER() OVER (
            PARTITION BY a.auditable_id
            ORDER BY a.changed_at ASC, a.id ASC
        ) AS rn
    FROM slq_main.audits a
    WHERE 
        a.auditable_type = 'App\\\\Http\\\\ApiCore\\\\Productos\\\\ProdProducto'
        AND a.event = 'created'
),

audit_upd AS (
    SELECT 
        a.auditable_id,
        COALESCE(a.usu_id, a.user_id) AS user_id,
        a.changed_at,
        ROW_NUMBER() OVER (
            PARTITION BY a.auditable_id
            ORDER BY a.changed_at DESC, a.id DESC
        ) AS rn
    FROM slq_main.audits a
    WHERE 
        a.auditable_type = 'App\\\\Http\\\\ApiCore\\\\Productos\\\\ProdProducto'
        AND a.event = 'updated'
),

adjuntos AS (
    SELECT 
        pa.id_producto,
        MAX(pa.fecha) AS fecha_adjunto
    FROM slq_main.prod_adjuntos pa
    WHERE pa.activo = 1 
      AND tipo = 154
    GROUP BY pa.id_producto
)

SELECT 
    p.id,
    p.created_at,
    COALESCE(au.changed_at, p.updated_at) AS updated_at,
    COALESCE(u_crea.name, u_reg.name) AS usuario_crea,
    u_upd.name AS usuario_actualiza,
    adj.fecha_adjunto

FROM slq_main.prod_productos p

LEFT JOIN audit_crea ac
    ON ac.auditable_id = p.id AND ac.rn = 1
LEFT JOIN slq_main.sys_users u_crea
    ON u_crea.id = ac.user_id

LEFT JOIN slq_main.sys_users u_reg
    ON u_reg.id = p.usuario_registro

LEFT JOIN audit_upd au
    ON au.auditable_id = p.id AND au.rn = 1
LEFT JOIN slq_main.sys_users u_upd
    ON u_upd.id = au.user_id

LEFT JOIN adjuntos adj
    ON adj.id_producto = p.id

WHERE 
    p.modulo_creacion = 'productos'
    AND p.bloqueo IN (1, 2)
    AND p.activo = 1
    AND p.created_at >= '2023-04-01'
""")

with engine_srv.connect() as conn:
    productos = pd.read_sql(query, conn)

productos


,id,created_at,updated_at,usuario_crea,usuario_actualiza,fecha_adjunto
0,39392,2023-04-02 19:16:08,2023-04-02 19:24:23,Danny Garizabalo,Danny Garizabalo,2019-04-27
1,39393,2023-04-02 19:31:27,2023-12-25 22:04:30,Danny Garizabalo,Michael Ramirez,2022-06-13
2,39394,2023-04-02 19:45:22,2023-04-02 19:52:42,Danny Garizabalo,Danny Garizabalo,2019-09-01
3,39395,2023-04-02 19:58:40,2023-04-02 20:02:46,Danny Garizabalo,Danny Garizabalo,2020-04-01
4,39396,2023-04-02 20:16:18,2023-04-02 20:20:05,Danny Garizabalo,Danny Garizabalo,2019-10-08
...,...,...,...,...,...,...
15610,69405,2025-12-02 16:15:37,2025-12-04 14:41:28,Cristian Camilo Gómez Pineda,None,2023-11-09
15611,69406,2025-12-02 16:36:47,2025-12-02 16:37:14,Cristian Camilo Gómez Pineda,Cristian Camilo Gómez Pineda,2025-02-04
15612,69407,2025-12-02 16:53:28,2025-12-02 16:54:03,Cristian Camilo Gómez Pineda,Cristian Camilo Gómez Pineda,2024-02-16
15613,69409,2025-12-02 17:27:08,2025-12-02 17:27:33,Cristian Camilo Gómez Pineda,Cristian Camilo Gómez Pineda,2024-03-12


In [36]:
productos.to_csv("productos.csv")

# Consultoría